In [ ]:
import numpy as np
from utility import *
from keras.utils.np_utils import to_categorical
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from scipy.stats import randint as sp_randint
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import hamming_loss, accuracy_score, make_scorer, jaccard_similarity_score, f1_score

In [ ]:
np.random.seed(2)

In [ ]:
#load training data
X_train, y_train, C = read_feats(layer=24, stride = 1, data_set='esc')
print(X_train.shape)
print(y_train.shape)

In [ ]:
X_train, X_test, labels_train, labels_test = train_test_split(X_train, y_train, test_size=0.2, random_state=2)
labels_train = labels_train.ravel()
labels_test = labels_test.ravel()

print('Split: ' + str(X_train.shape[0]) + '-' + str(X_test.shape[0]))
print(X_train.shape)
print(labels_train.shape)
X_train, labels_train = unfold_features(X_train, labels_train)
# print("Unfold Training shape")
# print(X_train.shape)
# print(labels_train.shape)
# print(labels_train)
# print("Test shape")
# print(X_test.shape)
# print(labels_test.shape)
# print("Keras shape")
labels_train = to_categorical(labels_train, num_classes=10)
labels_test = to_categorical(labels_test, num_classes=10)
# print(labels_train.shape)
# print(labels_test.shape)
# print(labels_train)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers

called = False

def create_model(hidden_layer_sizes=[(64, 'tanh'), (32, 'tanh')], lr=0.01, beta_1=0.8, beta_2=0.99, epsilon=1e-08, decay=0.0, reg=0.01, *param):
    # create model
    model = Sequential()
    for i, layer_param in enumerate(hidden_layer_sizes):
        # first layer
        if i == 0:
            model.add(Dense(layer_param[0], activation=layer_param[1], kernel_regularizer=regularizers.l1_l2(reg), input_dim=1024))
        else:
            model.add(Dense(layer_param[0], activation=layer_param[1], kernel_regularizer=regularizers.l1_l2(reg)))
#         model.add(Dropout(0.1))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(lr, beta_1, beta_2, epsilon, decay), *param)
    return model

In [ ]:
# Optimize the parameters by cross-validation
parameters = {
    'hidden_layer_sizes': [
#         [(32, 'tanh'), (16, 'tanh')],
        [(64, 'relu'), (32, 'relu')],
#         [(64, 'tanh'), (16, 'tanh')],
#         [(32, 'selu'), (32, 'selu'), (16, 'selu')]
    ],
#     'reg': [0.01, 0.001],
#     'beta_1': [0.8],
#     'beta_2': [0.99],
#     'epsilon': [1e-08],
#     'lr': [0.02]
};

# Best result:
# Validation: 0.738 (+/-0.006), Training: 0.953  for {'beta_1': 0.8, 'beta_2': 0.99, 'decay': 0.0, 'epsilon': 1e-08, 'hidden_layer_sizes': [(64, 'tanh'), (32, 'tanh')], 'lr': 0.01}

In [ ]:
# model = create_model()
# model.fit(X_train, labels_train, batch_size=8, epochs=10)

# model.summary()

# score = model.predict(np.array([X_train[0]]))
# print(score)
# print(sum(score[0]))

# Grid search object with KNN classifier.
clf = GridSearchCV(KerasClassifier(build_fn=create_model), param_grid=parameters, cv=2)
clf.fit(X_train, labels_train, batch_size=64, epochs=1000)

print("Best parameters set found on training set:")
print(clf.best_params_)

means_valid = clf.cv_results_['mean_test_score']
stds_valid = clf.cv_results_['std_test_score']
means_train = clf.cv_results_['mean_train_score']

print("Grid scores:")
for mean_valid, std_valid, mean_train, params in zip(means_valid, stds_valid, means_train, clf.cv_results_['params']):
    print("Validation: %0.3f (+/-%0.03f), Training: %0.3f  for %r" % (mean_valid, std_valid, mean_train, params))

In [ ]:
labels_predicted = fold_majority(clf, X_test, C)
# scores_predicted = fold_majority_score(clf, X_test, C)

labels_test_transformed = np.argmax(labels_test, axis=1)
print(labels_test_transformed)
print(labels_predicted)
# print(scores_predicted)

In [ ]:
print("Accuracy [%0.3f]" % ((labels_predicted == labels_test_transformed).mean()))
print("Hamming Loss [%0.3f]" % ((hamming_loss(labels_predicted, labels_test_transformed))))
print("Accuracy Score [%0.3f]" % ((accuracy_score(labels_predicted, labels_test_transformed))))
print("Similarity subset Score [%0.3f]" % ((jaccard_similarity_score(labels_predicted, labels_test_transformed))))
print("F1 Score [%0.3f]" % ((f1_score(labels_test_transformed, labels_predicted, average='macro'))))

In [ ]:
# probs = clf.predict_proba(X_test)
# print(probs)